In [1]:
import pandas as pd
import json
import gc
import networkx as nx
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import copy

In [2]:
def read_jsonfile(file , readfrom, readto,meta):
    print("read json")
    data=[]
    metaset=set(meta.index)
    metadic=meta.to_dict()
    num_of_review=0
    category_set=set()
    with open(file) as data_f:
        for line in data_f:
            num_of_review = num_of_review+1
            if num_of_review <readfrom:
                continue
            temp=json.loads(line)
            if temp['asin'] in metaset :
                if len(metadic['category'][temp['asin']])>0:
                    #print(metadic['category'][temp['asin']])
                    category_set.update(metadic['category'][temp['asin']])
                    data.append(temp)
            if num_of_review==readto:
                break
    df= pd.DataFrame(data)
    category_set.discard("Books")
    return df, data,category_set

In [3]:
def read_meta_jsonfile(file):
    print("read meta json")
    data=[]
    num_of_review=0
    with open(file) as data_f:
        for line in data_f:
            data.append(json.loads(line))
    df= pd.DataFrame(data,columns=['category','asin'])
    df.set_index('asin',inplace=True)

    return df

In [4]:
def remove_morethan_N_and_review2word(n,review_lis,data):
    print("read morethan N")
    countV = CountVectorizer(min_df=5)
    countV.fit(review_lis)
    print("after fit")
    cVoc=countV.vocabulary_
    inverse_cVoc={v:k for k,v in cVoc.items()}
    print("inv cvoc")
    formorethanN=np.zeros(shape=(len(cVoc)))
    review2word={}

        
    for i in range(len(data)):
        review2word[i]={}
    for i in range(int(len(review_lis)/batch_size)+1):
        if len(review_lis[i*batch_size:i*batch_size+batch_size])==0:
            break
        print('remove more than N :'+str(i)+'th')
        temp = countV.transform(review_lis[i*batch_size:i*batch_size+batch_size]).toarray()
        mat=temp.sum(axis=0)
        formorethanN=formorethanN+mat
        temp_pos=np.where(temp>0)
        for j in range(len(temp_pos[0])):
            if inverse_cVoc[temp_pos[1][j]] in review2word[i*batch_size + temp_pos[0][j]]:
                review2word[i*batch_size + temp_pos[0][j]][inverse_cVoc[temp_pos[1][j]]] += temp[temp_pos[0][j],temp_pos[1][j]]
            else:
                review2word[i*batch_size + temp_pos[0][j]][inverse_cVoc[temp_pos[1][j]]] = temp[temp_pos[0][j],temp_pos[1][j]]        
    del_lis=[]
    for i in range(len(formorethanN)):
        if formorethanN[i]>n:
            del_lis.append(inverse_cVoc[i])
    return del_lis ,review2word

In [5]:
def index2word(idx):
    return inv_tfidf_voc[idx]

In [6]:
def ret_wset(del_lis , review_lis,para):
    print("return word set and review to word")
    global inv_tfidf_voc
    tfidf = TfidfVectorizer(stop_words=del_lis,min_df=5).fit(review_lis)
    inv_tfidf_voc = {v:k for k,v in tfidf.vocabulary_.items()}
    words=set()
    review2word=[]
    for i in range(int(len(review_lis)/batch_size)+1):
        forlessthan_tfidf=[]
        if len(review_lis[i*batch_size:i*batch_size+batch_size])==0:
            break
        print('make word set: '+str(i)+'th')
        temp = tfidf.transform(review_lis[i*batch_size:i*batch_size+batch_size]).toarray()
        temp_pos=np.where(temp>para)
        forupdate=list(map(index2word,temp_pos[1]))
        for j in range(len(forupdate)):
            words.add(forupdate[j])
    word_li=list(words)
    
    return word_li,review2word

In [7]:
def ret_user2cate(user_lis,cate_set,data,metadic):
    dic={}
    for ele in user_lis:
        dic[ele]={}
    for line in data:
        user = line['reviewerID']
        item = line['asin']
        for ele in metadic['category'][item]:
            if ele != 'Books':
                #print(ele,metadic['category'][item])
                #print( dic[item])
                if ele in dic[user]:
                    dic[user][ele]=dic[user][ele]+1
                else: 
                    dic[user][ele]=1
    return dic

In [8]:
def ret_item2word(item_lis,word_lis,r2w,data):
    dic={}
    word_set=set(word_lis)
    for ele in item_lis:
        dic[ele]={}
    for i in range(len(r2w)):
        cur_r2w=r2w[i].items()
        cur_item=data[i]['asin']
        #print(cur_r2w)
        for ele in cur_r2w:
            if ele[0] in word_set:
                if ele[0] in dic[cur_item]:
                    dic[cur_item][ele[0]] = dic[cur_item][ele[0]]+ele[1]
                else:
                    dic[cur_item][ele[0]] = ele[1]

    return dic

In [9]:
def ret_user2word(user_lis,word_lis,r2w,data):
    dic={}
    word_set=set(word_lis)
    for ele in user_lis:
        dic[ele]={}
    for i in range(len(r2w)):
        cur_r2w=r2w[i].items()
        cur_id =data[i]["reviewerID"]
        for ele in cur_r2w:
            if ele[0] in word_set:
                if ele in dic[cur_id]:
                    dic[cur_id][ele[0]] += ele[1]
                else:
                    dic[cur_id][ele[0]] = ele[1]
    return dic
    

In [10]:
inv_tfidf_voc=None
meta=read_meta_jsonfile('meta_Books.json')
file='Books_5.json'


read meta json


In [11]:
df,data, category_set= read_jsonfile(file,0,2000000,meta) #a줄부터 b줄까지
print("number of review: "+str(len(data)))

read json
number of review: 1294652


In [12]:
batch_size=20000
asin_lis = list(df['asin'].drop_duplicates())
ID_lis = list(df['reviewerID'].drop_duplicates())
review_lis =list(df['reviewText'])
category_lis=list(category_set)
meta_dic=meta.to_dict()
del_lis ,review2word= remove_morethan_N_and_review2word(45000,review_lis,data)# n개 이하는 stopword로
#print(del_lis)

read morethan N
after fit
inv cvoc
remove more than N :0th
remove more than N :1th
remove more than N :2th
remove more than N :3th
remove more than N :4th
remove more than N :5th
remove more than N :6th
remove more than N :7th
remove more than N :8th
remove more than N :9th
remove more than N :10th
remove more than N :11th
remove more than N :12th
remove more than N :13th
remove more than N :14th
remove more than N :15th
remove more than N :16th
remove more than N :17th
remove more than N :18th
remove more than N :19th
remove more than N :20th
remove more than N :21th
remove more than N :22th
remove more than N :23th
remove more than N :24th
remove more than N :25th
remove more than N :26th
remove more than N :27th
remove more than N :28th
remove more than N :29th
remove more than N :30th
remove more than N :31th
remove more than N :32th
remove more than N :33th
remove more than N :34th
remove more than N :35th
remove more than N :36th
remove more than N :37th
remove more than N :38th


In [13]:
t_word_lis,aa=ret_wset(del_lis,review_lis,0.1)

return word set and review to word
make word set: 0th
make word set: 1th
make word set: 2th
make word set: 3th
make word set: 4th
make word set: 5th
make word set: 6th
make word set: 7th
make word set: 8th
make word set: 9th
make word set: 10th
make word set: 11th
make word set: 12th
make word set: 13th
make word set: 14th
make word set: 15th
make word set: 16th
make word set: 17th
make word set: 18th
make word set: 19th
make word set: 20th
make word set: 21th
make word set: 22th
make word set: 23th
make word set: 24th
make word set: 25th
make word set: 26th
make word set: 27th
make word set: 28th
make word set: 29th
make word set: 30th
make word set: 31th
make word set: 32th
make word set: 33th
make word set: 34th
make word set: 35th
make word set: 36th
make word set: 37th
make word set: 38th
make word set: 39th
make word set: 40th
make word set: 41th
make word set: 42th
make word set: 43th
make word set: 44th
make word set: 45th
make word set: 46th
make word set: 47th
make word set: 

In [14]:
###del_lis에 명사제외 단어를 전부 추가한다... 혹은 특징이라보기힘든단어들을 제거한다
import nltk
nltk.download('averaged_perceptron_tagger')

pos_lis = nltk.tag.pos_tag(t_word_lis)
filter_by_pos = set()
for w, p in pos_lis:
    if p in ['NN', 'NNS', 'NNP', 'NNPS']:
        filter_by_pos.add(w)

word_lis = list(filter_by_pos)
####

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/dilab/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [15]:
id2word=ret_user2word(ID_lis,word_lis,review2word,data)
item2word=ret_item2word(asin_lis,word_lis,review2word,data)
user2cate=ret_user2cate(ID_lis,category_set,data,meta_dic)

In [16]:
#그래프 생성
G=nx.Graph()
for ele in asin_lis:
    G.add_node(ele)
for ele in ID_lis:
    G.add_node(ele)
for ele in word_lis:
    G.add_node(ele)
for ele in category_lis:
    G.add_node(ele)
total_pos_w=0
total_pos_count=0
total_neg_w=0
total_neg_count=0
n= []

for ele in data:
    n.append(float(ele['overall']))
        

In [17]:
temp=np.array(n)
mean=np.mean(temp)
minn=9999999
maxn=0
for ele in data:
    if float(ele['overall'])>3:
        #w='positive'
        w=1-(float(ele['overall'])-mean)/4
        #w=0.1
        total_pos_count+=1
        total_pos_w+=w
        
    elif float(ele['overall'])<3:
        #w="negative"
        w=1-(float(ele['overall'])-mean)/4
        #w=2
        total_neg_count+=1
        total_neg_w+=w
    else :
        #w="neutral"   
        w=1
    G.add_edge(ele['reviewerID'],ele['asin'],weight=w)

#frequency 로
id_pos_rate=0.2
asin_pos_rate=0.2
maxw=0
minw=999999999
for ids in ID_lis:
    #print(id2word[ids])
    sorted_dic=sorted(id2word[ids].items(),key=lambda x:x[1],reverse=True)
    splitnum=int(len(sorted_dic)*id_pos_rate)
    for i in range(splitnum):
        #G.add_edge(ids,sorted_dic[i][0],weight='p') 
        G.add_edge(ids,sorted_dic[i][0],weight = 0.6850359109423767)
    for i in range(splitnum,len(sorted_dic)):
        #G.add_edge(ids,sorted_dic[i][0],weight='n') 
        G.add_edge(ids,sorted_dic[i][0],weight = 1.6449883206158211) 

for asi in asin_lis:        
    sorted_asin_dic=sorted(item2word[asi].items(),key=lambda x:x[1],reverse=True)
    #print(sorted_asin_dic)
    splitnum=int(len(sorted_asin_dic)*asin_pos_rate)
    #print(split)
    for i in range(splitnum):
        #G.add_edge(asi,sorted_asin_dic[i][0],weight='pos') 
        G.add_edge(asi,sorted_asin_dic[i][0],weight=0.6850359109423767) 
    for i in range(splitnum,len(sorted_asin_dic)):
        #G.add_edge(asi,sorted_asin_dic[i][0],weight='neg')
        G.add_edge(asi,sorted_asin_dic[i][0],weight=1.6449883206158211)
    
for asi in asin_lis:
    for ele in meta_dic['category'][asi]:
        if ele in category_set:
            G.add_edge(ele,asi,weight=0.5)

for ids in ID_lis:
    sorted_id_cate=sorted(user2cate[ids].items(),key=lambda x:x[1],reverse=True)
    total=0
    
    for i in range(len(sorted_id_cate)):
        total+=sorted_id_cate[i][1]
    
    for i in range(len(sorted_id_cate)):
        w= 1-((sorted_id_cate[i][1])/total)
        
        total_pos_w+=w
        total_pos_count+=1
        if w>maxw:
            maxw=w
        if w<minw:
            minw=w
        
        G.add_edge(ids,sorted_id_cate[i][0],weight= w)

'Name: \nType: Graph\nNumber of nodes: 479752\nNumber of edges: 36896211\nAverage degree: 153.8137'

In [18]:
from collections import deque
from networkx.generators.small import krackhardt_kite_graph

In [19]:
our_graph=G

In [20]:
import sys
from heapq import heappush, heappop
from collections import defaultdict
import json

count_t = 0
inf = 999
start = 'A26PRNQYA3Y662'
end = '0307932540'
k = 100
dp_dict = defaultdict(list)
for i in our_graph.nodes:
    dp_dict[i] = [[inf, []] for _ in range(k)]
heap = []

def dijkstra(start, end, count_t):
    heappush(heap, [0, start, []])
    print("start!",heap)
    dp_dict[start][0] = [0,[]]
    while heap:
        if count_t%1000000 == 0:
            print(count_t, end=' ')
        count_t += 1
        w, n, r = heappop(heap)
        r = r + [n]
        for n_n, wei in our_graph[n].items():
            n_w = 1 + w     
            if n_w < dp_dict[n_n][k-1][0] and n_w > 0:
                dp_dict[n_n][k-1] = [n_w, r+[n_n]]
                dp_dict[n_n].sort(key = lambda x:x[0]) 
                heappush(heap, [n_w, n_n, r])

## 탐색 ##
dijkstra(start, end, count_t)

##  distance 출력 ##
print('\n')
for i in dp_dict[end]:
    print("weight: ",i[0])
    print("route: ",i[1])
    print()
with open(start+'->'+end+'__distance__'+'.json','w') as f:
        json.dump(dp_dict[end], f)

## weight 로 sort ##
for i in dp_dict[end]:
    i[0] = 0
    for j in range(len(i[1])-1):
        i[0] += our_graph[i[1][j]][i[1][j+1]]['weight']
dp_dict[end].sort(key = lambda x:x[0])   

##  weight 출력 ##
print('\n')
for i in dp_dict[end]:
    print("weight: ",i[0])
    print("route: ",i[1])
    print()
with open(start+'->'+end+'__weight__'+'.json','w') as f:
        json.dump(dp_dict[end], f)

start! [[0, 'A26PRNQYA3Y662', []]]
0 1000000 2000000 3000000 4000000 5000000 6000000 7000000 8000000 9000000 10000000 11000000 12000000 13000000 14000000 15000000 16000000 17000000 18000000 19000000 20000000 21000000 22000000 23000000 24000000 25000000 26000000 27000000 28000000 29000000 30000000 31000000 32000000 33000000 34000000 35000000 36000000 37000000 38000000 39000000 40000000 41000000 42000000 43000000 44000000 45000000 46000000 47000000 

weight:  2
route:  ['A26PRNQYA3Y662', 'Literature &amp; Fiction', '0307932540']

weight:  2
route:  ['A26PRNQYA3Y662', 'United States', '0307932540']

weight:  2
route:  ['A26PRNQYA3Y662', 'december', '0307932540']

weight:  3
route:  ['A26PRNQYA3Y662', 'Literature &amp; Fiction', 'A1004NB4ADJVZ2', '0307932540']

weight:  3
route:  ['A26PRNQYA3Y662', 'United States', 'A1004NB4ADJVZ2', '0307932540']

weight:  3
route:  ['A26PRNQYA3Y662', 'Literature &amp; Fiction', 'A1011OGVLL3WUE', '0307932540']

weight:  3
route:  ['A26PRNQYA3Y662', 'United

In [21]:
our_graph=G
import sys
from heapq import heappush, heappop
from collections import defaultdict
import json

count_t = 0
inf = 999
start = 'A26PRNQYA3Y662'
end = '0307932540'
k = 10
dp_dict = defaultdict(list)
for i in our_graph.nodes:
    dp_dict[i] = [[inf, []] for _ in range(k)]
heap = []
# with open(start+'->'+end+'__distance__'+'.json','w') as f:
#         json.dump(dp_dict[end], f)

In [33]:
# our_graph=G
# for i in dp_dict[end]:
#     i[0] = 0
#     for j in range(len(i[1])-1):
#         i[0] += our_graph[i[1][j]][i[1][j+1]]['weight']
with open(start+'->'+end+'.json','r',encoding="UTF-8") as f:
    dp_dict[end] = json.load(f)
    
dp_dict[end].sort(key = lambda x:x[0])   

##  weight 출력 ##
print('\n')
for i in dp_dict[end]:
    print("weight: ",i[0])
    print("route: ")
    for j in range(len(i[1])-1):
        temp1=i[1][j]
        temp2=i[1][j+1]
        
        if temp1 in asin_lis: 
            if temp2 in ID_lis: #item -> user
                rate = None
                for ele in data:
                    if ele['reviewerID']==temp2 and ele['asin']==temp1:
                        rate=ele['overall']
                print(temp1,"->",rate,"->",temp2)
            elif temp2 in category_lis: #item -> cate
                print(temp1,"->",temp2)
            elif temp2 in word_lis:    # item -> word
                print(temp1,"->",item2word[temp1][temp2],"->",temp2)
            
        elif temp1 in ID_lis:
            if temp2 in asin_lis:  # user -> item
                rate = None
                for ele in data:
                    if ele['reviewerID']==temp2 and ele['asin']==temp1:
                        rate=ele['overall']
                print(temp1,"->",rate,"->",temp2)
            elif temp2 in category_lis:  # user -> cate
                print(temp1,"->",user2cate[temp1][temp2],"->",temp2)
            elif temp2 in word_lis:    # user -> word
                print(temp1,"->",id2word[temp1][temp2],"->",temp2)
            
        elif temp1 in word_lis:
            if temp2 in ID_lis:      # word -> user
                print(temp1,"->",id2word[temp2][temp1],"->",temp2)
            elif temp2 in asin_lis:  # word -> item
                 print(temp1,"->",item2word[temp2][temp1],"->",temp2)
            
        elif temp1 in category_lis:
            if temp2 in ID_lis:    # cate -> user
                print(temp1,"->",user2cate[temp2][temp1],"->",temp2)
            elif temp2 in asin_lis:  # cate -> item
                print(temp1,"->",temp2)
                
    print()
with open(start+'->'+end+'__weight__'+'.json','w') as f:
        json.dump(dp_dict[end], f)



weight:  0.2
route: 
A26PRNQYA3Y662 -> 1 -> Literature &amp; Fiction
Literature &amp; Fiction -> 0307932540

weight:  0.2
route: 
A26PRNQYA3Y662 -> 1 -> United States
United States -> 0307932540

weight:  0.30000000000000004
route: 
A26PRNQYA3Y662 -> 1 -> Literature &amp; Fiction
Literature &amp; Fiction -> 2 -> A1004NB4ADJVZ2
A1004NB4ADJVZ2 -> None -> 0307932540

weight:  0.30000000000000004
route: 
A26PRNQYA3Y662 -> 1 -> United States
United States -> 2 -> A1004NB4ADJVZ2
A1004NB4ADJVZ2 -> None -> 0307932540

weight:  0.30000000000000004
route: 
A26PRNQYA3Y662 -> 1 -> Literature &amp; Fiction
Literature &amp; Fiction -> 1 -> A1011OGVLL3WUE
A1011OGVLL3WUE -> None -> 0307932540

weight:  0.30000000000000004
route: 
A26PRNQYA3Y662 -> 1 -> United States
United States -> 1 -> A1011OGVLL3WUE
A1011OGVLL3WUE -> None -> 0307932540

weight:  0.30000000000000004
route: 
A26PRNQYA3Y662 -> 1 -> Literature &amp; Fiction
Literature &amp; Fiction -> 1 -> A102UTTLGQ56WT
A102UTTLGQ56WT -> None -> 030